# BÁO CÁO THỐNG KÊ MÔ TẢ CHI TIẾT DỮ LIỆU COVID-19
## Phân tích theo Quốc gia (Country)


## 1. Mục tiêu phân tích

Mục tiêu của notebook này là thực hiện **thống kê mô tả chi tiết** cho bộ dữ liệu COVID-19.

Các nội dung chính bao gồm:
- Nhóm dữ liệu theo **4 Quốc gia (Country)**
- Tính các chỉ số thống kê mô tả
- Phân tích mức độ lây nhiễm, tử vong và tăng trưởng dịch bệnh

Phương pháp thống kê sử dụng:
- Mean (Giá trị trung bình)
- Median (Trung vị)
- Standard Deviation (Độ lệch chuẩn)
- Quartile Q1, Q3
- Min / Max


## 2. Import thư viện

Sử dụng các thư viện tiêu chuẩn trong phân tích dữ liệu:
- `pandas`: xử lý dữ liệu dạng bảng
- `numpy`: hỗ trợ tính toán thống kê


In [1]:
import pandas as pd
import numpy as np

## 3. Đọc dữ liệu

Dữ liệu được lưu dưới dạng file `.csv.gz`.  
Hàm `read_csv()` của pandas tự động giải nén file khi đọc.


In [2]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/03_4_country_population_summary.csv.gz")

## 4. Khám phá dữ liệu ban đầu

Xem một số dòng đầu tiên của bộ dữ liệu để:
- Hiểu cấu trúc dữ liệu
- Xác định ý nghĩa của từng cột


In [3]:
df.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,New_deaths,Cumulative_cases,Cumulative_deaths,New_cases_MA7,New_deaths_MA7,Growth_Rate
0,2020-01-04,CN,China,WPR,1.0,0.0,1,0,1.000000,0.0,0.0
1,2020-01-05,CN,China,WPR,0.0,0.0,1,0,0.500000,0.0,-100.0
2,2020-01-06,CN,China,WPR,3.0,0.0,4,0,1.333333,0.0,0.0
3,2020-01-07,CN,China,WPR,0.0,0.0,4,0,1.000000,0.0,-100.0
4,2020-01-08,CN,China,WPR,0.0,0.0,4,0,0.800000,0.0,0.0


### Nhận xét

- Mỗi dòng dữ liệu biểu diễn **một quốc gia tại một thời điểm (ngày báo cáo)**
- Dữ liệu có cấu trúc rõ ràng, thuận tiện cho phân tích chuỗi thời gian

## 5. Kiểm tra kiểu dữ liệu và số lượng quan sát

Mục đích:
- Xác định kiểu dữ liệu của từng biến
- Kiểm tra giá trị thiếu (missing values)


In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8380 entries, 0 to 8379
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date_reported      8380 non-null   object 
 1   Country_code       8380 non-null   object 
 2   Country            8380 non-null   object 
 3   WHO_region         8380 non-null   object 
 4   New_cases          8380 non-null   float64
 5   New_deaths         8380 non-null   float64
 6   Cumulative_cases   8380 non-null   int64  
 7   Cumulative_deaths  8380 non-null   int64  
 8   New_cases_MA7      8380 non-null   float64
 9   New_deaths_MA7     8380 non-null   float64
 10  Growth_Rate        8380 non-null   float64
dtypes: float64(5), int64(2), object(4)
memory usage: 720.3+ KB


### Nhận xét

- Không tồn tại giá trị bị thiếu
- Các biến định lượng phù hợp để thống kê mô tả


## 6. Lựa chọn các biến định lượng để phân tích


In [5]:
numeric_cols = [
    "New_cases",
    "New_deaths",
    "Cumulative_cases",
    "Cumulative_deaths",
    "Growth_Rate"
]

### Nhận xét

Các biến được chọn phản ánh đầy đủ:
- Quy mô lây nhiễm
- Mức độ tử vong
- Tốc độ tăng trưởng dịch bệnh


## 7. Thống kê mô tả tổng quát toàn bộ dữ liệu


In [6]:
df[numeric_cols].describe()

,New_cases,New_deaths,Cumulative_cases,Cumulative_deaths,Growth_Rate
count,8.380000e+03,8380.000000,8.380000e+03,8.380000e+03,8380.000000
mean,4.608380e+04,353.856862,4.030372e+07,3.426552e+05,18.843505
std,2.207762e+05,856.997517,3.999091e+07,4.052244e+05,525.667173
min,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,-100.000000
25%,8.357143e+00,0.000000,3.318312e+06,3.075375e+04,-8.024802
50%,4.735000e+02,2.714286,3.073125e+07,1.222480e+05,0.000000
75%,4.589475e+04,365.250000,9.474325e+07,5.336230e+05,5.117022
max,6.966046e+06,44047.000000,1.034368e+08,1.228289e+06,36542.268041


### Nhận xét

- Giá trị Max rất lớn so với Median → dữ liệu **lệch phải**
- Có sự xuất hiện của **outlier**
- Phân phối dữ liệu không tuân theo phân phối chuẩn


## 8. Nhóm dữ liệu theo Quốc gia (Country)

Mỗi nhóm đại diện cho chuỗi dữ liệu theo thời gian của một quốc gia


In [7]:
group_country = df.groupby("Country")[numeric_cols]

## 9. Thống kê mô tả chi tiết theo Quốc gia


In [8]:
country_stats = group_country.agg(
    count=("New_cases", "count"),

    new_cases_mean=("New_cases", "mean"),
    new_cases_median=("New_cases", "median"),
    new_cases_std=("New_cases", "std"),
    new_cases_min=("New_cases", "min"),
    new_cases_q1=("New_cases", lambda x: x.quantile(0.25)),
    new_cases_q3=("New_cases", lambda x: x.quantile(0.75)),
    new_cases_max=("New_cases", "max"),

    deaths_mean=("New_deaths", "mean"),
    deaths_median=("New_deaths", "median"),
    deaths_max=("New_deaths", "max"),

    growth_mean=("Growth_Rate", "mean"),
    growth_std=("Growth_Rate", "std")
)

### Nhận xét

- Một số quốc gia có Median thấp nhưng Max rất cao → dịch bùng phát theo **đợt**
- Độ lệch chuẩn lớn cho thấy mức độ biến động mạnh


## 10. Sắp xếp quốc gia có số ca nhiễm mới trung bình cao nhất


In [10]:
country_stats.sort_values(
    by="new_cases_mean",
    ascending=False
)

,count,new_cases_mean,new_cases_median,new_cases_std,new_cases_min,new_cases_q1,new_cases_q3,new_cases_max,deaths_mean,deaths_median,deaths_max,growth_mean,growth_std
Country,,,,,,,,,,,,,
United States of America,2095,109823.659427,93260.0,108672.289657,1.0,58599.500000,96239.0,1265520.0,1081.132697,794.0,5061.0,5.809861,45.070038
China,2095,47437.594749,54.0,415831.239701,0.0,0.000000,572.0,6966046.0,58.473747,0.0,44047.0,38.125870,937.447553
India,2095,21525.470883,602.0,56296.444250,0.0,62.428571,16077.0,414188.0,255.197613,5.0,6148.0,6.429039,83.291974
Viet Nam,2095,5548.482578,4.0,28808.346698,0.0,0.000000,530.5,454212.0,20.623389,0.0,804.0,25.009250,466.035237


### Nhận xét

- Các quốc gia này chịu ảnh hưởng nặng của dịch bệnh
- Trung bình cao đi kèm độ biến động lớn


## 11. Kết luận

- Dữ liệu COVID-19 biến động mạnh giữa các quốc gia
- Thống kê mô tả theo nhóm giúp đánh giá hiệu quả mức độ ảnh hưởng
- Đây là nền tảng cho phân tích nâng cao và trực quan hóa dữ liệu
